In [10]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.cm as cm # импортируем цветовые схемы, чтобы рисовать графики.
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn import linear_model
from scipy.special import erfc
from scipy.signal import correlate
from scipy.fftpack import fft
import statsmodels.api as sm
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
X_train = pd.read_csv('data_public/x_train.csv', header=None)
Y_train = pd.read_csv('data_public/y_train.csv', header=None)
Y_train.columns = ['target']
X_test = pd.read_csv('data_public/x_test.csv', header=None)

In [3]:
def r(a,b):
    if (a == b):
        return 0
    else:
        return 1
    
def switchness(seq):
    return np.sum([r(seq[i], seq[i+1]) for i in range(len(seq)-1)]) +1

assert(switchness([1,0,0,1,1,0,1,0,1,1])==7)

In [4]:
def longest_seq(seq):
    return np.max([len(sub) for sub in "".join([str(i) for i in seq]).split('0')])
def longest_seq_sum(seq):
    return np.sum([len(sub) for sub in "".join([str(i) for i in seq]).split('0')])
def longest_seq_mean(seq):
    return np.mean([len(sub) for sub in "".join([str(i) for i in seq]).split('0')])
assert(longest_seq([1,1,1,0,1,0,1,1])==3)

In [5]:
arr = [1,0,1,0,1,0,1,0]
arr2= [1,1,1,1,1,1,1,1]
np.max(np.abs(fft(arr2)))

8.0

In [62]:
def extract_features(data):
    word = data.apply(lambda row: ''.join([str(c) for c in row]), axis=1)
    d = pd.DataFrame(index=data.index)
    d['freq_bitwise_test'] = data.apply(lambda row: erfc((2.0*np.sum(row) - row.size) / np.sqrt(row.size)), axis=1)
    d['switchness'] = data.apply(lambda row: switchness(row), axis=1)
    d['longest_seq'] = data.apply(lambda row: longest_seq(row), axis=1)
    d['longest_seq_sum'] = data.apply(lambda row: longest_seq_sum(row), axis=1)
    d['longest_seq_mean'] = data.apply(lambda row: longest_seq_mean(row), axis=1)
    # d['kurtosis'] = data.apply(lambda row: stats.kurtosis(row), axis=1)
    # d['normal_stat'] = data.apply(lambda row: stats.normaltest(row)[1], axis=1)
    d['mean'] = data.apply(lambda row: np.mean(row), axis=1)
    d['entropy'] = data.apply(lambda row: stats.entropy(row), axis=1)
    
    vec = CountVectorizer(analyzer='char', ngram_range=(6,8))
    bag = pd.DataFrame(vec.fit_transform(word).toarray())
    
    d['adfuller'] = data.apply(lambda row: sm.tsa.stattools.adfuller(row)[1], axis=1)
    d['student'] = data.apply(lambda row: stats.ttest_1samp(row,0)[1], axis=1)
    return pd.concat([d.fillna(0), bag], axis=1)


In [63]:
d = extract_features(X_train)
d['target'] = Y_train
X = d.drop(['target'], axis=1)
d.head()

/usr/local/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:660: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/usr/local/lib/python3.5/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


,freq_bitwise_test,switchness,longest_seq,longest_seq_sum,longest_seq_mean,mean,entropy,adfuller,student,0,...,439,440,441,442,443,444,445,446,447,target
0,0.605577,15,5,16,1.066667,0.533333,2.772589,1.973197e-05,3.108541e-06,0,...,0,1,0,0,0,0,0,0,0,1
1,0.605577,20,3,16,1.066667,0.533333,2.772589,2.670386e-10,3.108541e-06,0,...,0,0,0,0,0,0,0,0,0,1
2,1.000000,19,3,15,0.937500,0.500000,2.708050,4.944840e-09,8.699475e-06,0,...,0,0,0,0,0,0,0,0,0,1
3,0.301700,20,3,17,1.214286,0.566667,2.833213,2.953531e-01,1.032992e-06,0,...,0,0,0,0,0,0,0,0,0,0
4,0.038867,19,4,19,1.583333,0.633333,2.944439,2.064574e-10,8.716303e-08,0,...,0,0,0,0,0,0,0,0,0,2


In [64]:
def linear(X, y):
    clf = linear_model.SGDClassifier(loss='log')
    params = {
        'alpha': [0.0001],
        'n_iter': [100]
    }
    grid_search = GridSearchCV(clf, param_grid=params, cv=StratifiedKFold(5, random_state=42), scoring='accuracy')
    grid_search.fit(X, y)
    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    return grid_search

scaler = StandardScaler()
linear_clf = linear(scaler.fit_transform(X), Y_train.target)

Best score: 0.5428571428571428
Best parameters: {'n_iter': 100, 'alpha': 0.0001}


In [65]:
def knn(X, y):
    clf = KNeighborsClassifier()
    params = {
        'n_neighbors': [5, 8, 9, 10, 11, 12, 13, 14,15,16,17,18,19,20], 
        'metric': ['hamming', 'jaccard', 'minkowski'],
        'weights': ['uniform', 'distance']}
    grid_search = GridSearchCV(clf, param_grid=params, cv=StratifiedKFold(5, random_state=42), scoring='accuracy')
    grid_search.fit(X, y)
    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    return grid_search

knn_clf = knn(X, Y_train.target)

Best score: 0.6523809523809524
Best parameters: {'n_neighbors': 20, 'weights': 'distance', 'metric': 'minkowski'}


In [66]:
xgb_clf = xgb.XGBClassifier(n_estimators=100)
scores = cross_val_score(xgb_clf, X, Y_train.target, cv=StratifiedKFold(5, random_state=42), scoring='accuracy')
print("Score: {} (+/- {})".format(scores.mean(), scores.std() ** 2))

Score: 0.5900498338870432 (+/- 0.006733794819532281)


In [67]:
clf = VotingClassifier([("xgb", xgb_clf), ("knn", knn_clf), ("lin", linear_clf)], voting='soft')
scores = cross_val_score(clf, X, Y_train.target, cv=StratifiedKFold(5, random_state=42), scoring='accuracy')
print("Score: {} (+/- {})".format(scores.mean(), scores.std() ** 2))

/usr/local/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/usr/local/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/usr/local/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/usr/local/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Score: 0.5511129568106312 (+/- 0.005378406541993036)


/usr/local/lib/python3.5/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [232]:
clf.fit(X, Y_train.target)

VotingClassifier(estimators=[('xgb', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       sc...re_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0))],
         n_jobs=1, voting='hard', weights=None)

In [68]:
d_test = extract_features(X_test)
d_test.head()

/usr/local/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:660: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


,freq_bitwise_test,switchness,longest_seq,longest_seq_sum,longest_seq_mean,mean,entropy,adfuller,student,0,...,438,439,440,441,442,443,444,445,446,447
0,0.121335,16,4,18,1.384615,0.600000,2.890372,6.120728e-01,3.154131e-07,0,...,0,0,0,0,0,0,0,0,0,0
1,1.878665,21,2,12,0.631579,0.400000,2.484907,5.169520e-01,1.349328e-04,0,...,0,0,0,0,0,0,0,0,0,0
2,1.698300,12,5,13,0.722222,0.433333,2.564949,2.561162e-03,5.689138e-05,2,...,0,0,0,0,0,0,0,0,0,0
3,1.394423,22,2,14,0.823529,0.466667,2.639057,6.260612e-09,2.286408e-05,0,...,0,0,0,0,0,0,0,0,0,0
4,0.301700,14,5,17,1.214286,0.566667,2.833213,2.364495e-07,1.032992e-06,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
pred = knn_clf.predict(d_test)
submission = pd.DataFrame(pred, index=X_test.index)
submission.to_csv('submission_3_knn_count.csv', header=False, index=False)